In [ ]:
import h5py
import os
import trimesh
import numpy as np
import h5py
from utils.trackedmesh_loader import TrackedMeshLoader
from utils.Blendshape import DeformationComponents
save_path = "./samples/"

In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()
date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

# Deformation components
- Apply following methhods on the tracked mashes from multiface dataset
1. PCA
2. MiniBatch Sparse PCA

# Load data matrix

In [ ]:
path_to_dataset = os.path.join(os.getcwd(), '../dataset/multiface/tracked_mesh/')
NofFrames = 5
tml = TrackedMeshLoader(path_to_dataset=path_to_dataset, ID=6795937, suffix='E0', mesh_loader="trimesh", num_samples_perExp=NofFrames)
dataMat, MEAN, cent_X = tml()

In [ ]:
dataMat.shape

In [ ]:
cent_X.shape

In [ ]:
centX_std = np.std(cent_X)

# Load Facemask

In [ ]:
from utils.Blendshape import FaceMask
from utils.pickel_io import dump_pckl, load_from_memory

# set the name of pickel file according to the mesh_loader
if tml.mesh_loader == "trimesh":
    pickel_fname = "FaceMask_sample_22012024_14_19_trimesh.pkl"
elif tml.mesh_loader == "original":
    pickel_fname = "FaceMask_30102023_09_40.pkl"

facemask = load_from_memory(path_to_memory = save_path, pickle_fname = pickel_fname)

In [ ]:
masked_cent_X = cent_X * facemask.bit_mask[None, :]
mesh = trimesh.load(os.path.join(save_path, "sample.obj"), force='mesh')
tris = np.asarray(mesh.faces)

dump if you want to save time from the second time

In [ ]:
# # dump the data matrix if you need to dump the matrix to save loading time
# from utils.Blendshape import ZeroMeanDefMatrix

# deformation_data = ZeroMeanDefMatrix(masked_cent_x = masked_cent_X, mean = tml.ave_neutralmesh_vertices, std = tml.centX_std, tris = tris)
# dd_pickel_fname = 'deformation_data_matrix_and_mean'+ '_' +date+'_'+time+'_'+tml.mesh_loader+'.pkl'
# dump_pckl(data = deformation_data, save_root= save_path, pickel_fname=dd_pickel_fname)

# Set the number of components

In [ ]:
Ncompos = 50
preScaleFactor = 1/tml.centX_std
N_cent_X = cent_X * preScaleFactor
masked_N_cent_X = masked_cent_X * preScaleFactor

# 1. PCA

In [ ]:
from sklearn.decomposition import PCA
D = masked_cent_X.shape[0]
pca = PCA(D)
pca.fit(N_cent_X.reshape(N_cent_X.shape[0], -1))

In [ ]:
Gamma = pca.components_.reshape(D, -1, 3) #right-hand side matrix of eigenvalue decomposition
Variance = pca.explained_variance_
Stds = np.sqrt(Variance)
Sigma = np.diag(pca.explained_variance_) #diagonal matrix which has eigenvectors in each diagonal entry
MEAN =  tml.ave_neutralmesh_vertices # neutral face mesh (vertex coodinates of neutral face)

In [ ]:
masked_PCA_dc = Gamma * facemask.bit_mask[None, :]

In [ ]:
PCA_gamma = Gamma
PCA_variance = Variance

In [ ]:
Gamma.shape

In [ ]:
pca_RMSE = np.sqrt(((masked_N_cent_X.reshape(masked_N_cent_X.shape[0], -1) - np.dot(Sigma, (masked_PCA_dc).reshape(masked_PCA_dc.shape[0], -1)))**2).mean())

In [ ]:
print(pca_RMSE)

In [ ]:
pca_sparsity = np.sum(np.sqrt((masked_PCA_dc[:Ncompos, :]**2).sum(axis = 2)))

In [ ]:
print(pca_sparsity)

In [ ]:
pca_sparsity_level = np.mean(masked_PCA_dc==0)
print(pca_sparsity_level)

# 2. MiniBatch SparsePCA

In [ ]:
from sklearn.decomposition import MiniBatchSparsePCA
mb_sparsePCA = MiniBatchSparsePCA(n_components=Ncompos, verbose=True)
est_MBSPCA = mb_sparsePCA.fit(N_cent_X.reshape(N_cent_X.shape[0], -1))

In [ ]:
C_mbspca = est_MBSPCA.components_.reshape(Ncompos, -1, 3) #right hand side V
W_mbspca = est_MBSPCA.transform(masked_cent_X.reshape(masked_cent_X.shape[0], -1)) #left hand side U

In [ ]:
masked_MBSPCA_dc = C_mbspca * facemask.bit_mask[None, :]

In [ ]:
MBSPCA_C = C_mbspca
MBSPCA_W = W_mbspca

In [ ]:
MBSPCA_C.shape

In [ ]:
mbspca_RMSE = np.sqrt(((masked_N_cent_X.reshape(masked_N_cent_X.shape[0], -1) - np.dot(W_mbspca, (masked_MBSPCA_dc).reshape(masked_MBSPCA_dc.shape[0], -1)))**2).mean())


In [ ]:
print(mbspca_RMSE)

In [ ]:
mbspca_sparsity = np.sum(np.sqrt((masked_MBSPCA_dc**2).sum(axis = 2)))

In [ ]:
print(mbspca_sparsity)

In [ ]:
mbspca_sparsity_level = np.mean(masked_MBSPCA_dc==0)
print(mbspca_sparsity_level)

# save deformation components

In [ ]:
pca_hdf5 = "tracked_mesh"+"_5perExp_"+tml.mesh_loader+"PCA_dcs.hdf5"
PCA_deformationCompos = DeformationComponents(dataMat=dataMat, faceMask=facemask.bit_mask[None, :], MEAN = MEAN, STD = centX_std, pcMat=PCA_gamma, coeffMat=Sigma, tris = tris, NofExp=dataMat.shape[0], NofFrame=5, NofVerts=int(dataMat.shape[1]))
PCA_deformationCompos.save_hdf5(path_to_save=path_to_dataset, fname = pca_hdf5)
pca_f = h5py.File(os.path.join(path_to_dataset, pca_hdf5), 'r')
list(pca_f.keys())
mbspca_hdf5 = "tracked_mesh"+"_5perExp_"+tml.mesh_loader+"MBSPCA_dcs.hdf5"
MBSPCA_deformationCompos = DeformationComponents(dataMat=dataMat, faceMask=facemask.bit_mask[None, :], MEAN = MEAN, STD=centX_std, pcMat=MBSPCA_C, coeffMat=MBSPCA_W, tris = tris, NofExp=dataMat.shape[0], NofFrame=5, NofVerts=int(dataMat.shape[1]))
MBSPCA_deformationCompos.save_hdf5(path_to_save=path_to_dataset, fname = mbspca_hdf5)
mbspca_f = h5py.File(os.path.join(path_to_dataset, mbspca_hdf5), 'r')
list(mbspca_f.keys())